In [ ]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Connect to Elasticsearch
es = Elasticsearch(hosts=["http://localhost:9200"])

# Load BERT Model for Embedding
model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate Embeddings for FAISS
corpus = ["Elasticsearch is a search engine.", "FAISS is used for semantic search."]
corpus_embeddings = model.encode(corpus)

# Build FAISS Index
dimension = corpus_embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(np.array(corpus_embeddings))

# Query Input
query = "What is semantic search?"
query_embedding = model.encode([query])

# FAISS Search
distances, indices = faiss_index.search(np.array(query_embedding), k=2)
faiss_results = [corpus[i] for i in indices[0]]

# Elasticsearch Search
es_query = {"query": {"match": {"content": query}}}
es_response = es.search(index="knowledge_base", body=es_query)
es_results = [hit["_source"]["content"] for hit in es_response["hits"]["hits"]]

# Combine Results
combined_results = faiss_results + es_results

# Generate Answer with T5
generator = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")
context = " ".join(combined_results)
input_text = f"question: {query} context: {context}"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output = generator.generate(input_ids, max_length=50)
print("Generated Answer:", tokenizer.decode(output[0], skip_special_tokens=True))
